In [2]:
import numpy as np
import pandas as pd
import random
import re

In [3]:
daftar_kata = list(pd.read_csv('lima huruf.csv', header=None)[0])
print(f"Banyak Kata : {len(daftar_kata)}")
daftar_kata

Banyak Kata : 6311


['abadi',
 'abaka',
 'abang',
 'abdas',
 'abece',
 'abian',
 'abjad',
 'abnus',
 'abrak',
 'abrar',
 'abras',
 'abrek',
 'absah',
 'absen',
 'abses',
 'absis',
 'abtar',
 'abuan',
 'acala',
 'acang',
 'acara',
 'acawi',
 'acung',
 'adaks',
 'adang',
 'adati',
 'adika',
 'adisi',
 'adnan',
 'adres',
 'advis',
 'aerob',
 'afair',
 'afdal',
 'afiat',
 'afiks',
 'afsun',
 'agama',
 'agami',
 'agogo',
 'agung',
 'ahkam',
 'ahmak',
 'ahwal',
 'ajaib',
 'ajang',
 'ajung',
 'akang',
 'akbar',
 'akene',
 'akhir',
 'akmal',
 'akrab',
 'aksen',
 'aksep',
 'akses',
 'aksis',
 'akson',
 'aktif',
 'aktor',
 'akwal',
 'alami',
 'alang',
 'alarm',
 'albas',
 'albit',
 'album',
 'alias',
 'alibi',
 'aling',
 'alkil',
 'altar',
 'alung',
 'alusi',
 'alwah',
 'amang',
 'ambah',
 'ambak',
 'ambal',
 'ambar',
 'ambek',
 'ambet',
 'ambil',
 'ambin',
 'amboi',
 'ambur',
 'ameba',
 'amina',
 'amino',
 'amorf',
 'ampai',
 'ampas',
 'ampat',
 'ampek',
 'ampel',
 'ampuh',
 'ampuk',
 'ampul',
 'ampun',
 'amput',


# Fungsi nebak

! = benar <br>
? = ada tapi salah tempat <br>
\* = tidak ada huruf tersebut


In [4]:
class Katla():
    def __init__(self, daftar_kata):
        self.daftar_kata = daftar_kata
        self.huruf_benar = [False, False, False, False, False]

    def kandidat(self, tebakan, pola, daf_kata):
        list_tanda_seru    = [idx for idx,huruf in enumerate(pola) if huruf=='!']
        list_tanda_tanya   = [idx for idx,huruf in enumerate(pola) if huruf=='?']
        list_tanda_bintang = [idx for idx,huruf in enumerate(pola) if huruf=='*']

        # Pola Regex
        pola_regex = ["."]*5
        for pos in list_tanda_seru:                 # huruf di tanda seru benar
            pola_regex[pos] = tebakan[pos] 
            self.huruf_benar[pos] = True   
        for pos in list_tanda_tanya:                # huruf di tanda tanya salah
            pola_regex[pos] = f"[^{tebakan[pos]}]"

        # Cari daftar kata sesuai pola regex
        pola_regex = r"".join(pola_regex)
        daf_kata =  [kata for kata in daf_kata if re.match(pola_regex, kata)]

        # Hilangkan kata yang ada huruf di list_tanda_bintang
        list_huruf_salah = [tebakan[pos] for pos in list_tanda_bintang]
        new_daf_kata = []
        for kata in daf_kata:
            tidak_ada_huruf_salah = True
            kata_tanpa_huruf_benar = [kata[pos] for pos in range(5) if not self.huruf_benar[pos]]
            for huruf_salah in list_huruf_salah:
                if huruf_salah in kata_tanpa_huruf_benar:
                    tidak_ada_huruf_salah = False; break
            if tidak_ada_huruf_salah:
                new_daf_kata.append(''.join(kata))
        daf_kata = new_daf_kata 

        # Cara kata yang ada huruf di tanda tanya
        for pos in list_tanda_tanya:
            huruf = tebakan[pos]
            daf_kata = [kata for kata in daf_kata if huruf in kata]
        
        return daf_kata

    def tebakan_selanjutnya(self, tebakan, pola, tampilkan_detail=False):
        self.daftar_kata = self.kandidat(tebakan, pola, self.daftar_kata)

        if tampilkan_detail:
            print(f"Banyak Kata : {len(self.daftar_kata)}\n{self.daftar_kata}\n\n")

        kata = random.choice(self.daftar_kata) if len(self.daftar_kata) != 0 else "TIDAK ADA KATA"

        return kata




In [5]:
class GameKatla():
    def __init__(self, daftar_kata):
        self.jumlah_tebakan = 0
        self.MENANG = "!!!!!"
        self.__jawaban = random.choice(daftar_kata)


    def tebak(self, tebakan):
        self.jumlah_tebakan += 1
        respon_pola = self.__evaluasi_tebakan(tebakan)

        if respon_pola == self.MENANG:                  # Sudah sama maka menang
            return ("selesai", self.jumlah_tebakan)
        if self.jumlah_tebakan == 6:                    # Sudah habis batas tebak dan masih salah
            return ("selesai", 10)
        

    def __evaluasi_tebakan(self, tebakan):
        list_jawaban = []
        respon_pola = ['*']*5
        for i in range(5):
            if self.__jawaban[i] == tebakan[i]:
                respon_pola[i] = '!'
            else:
                list_jawaban.append(self.__jawaban[i])
        
        for i in range(5):
            if respon_pola[i] == '!':
                continue
            if tebakan[i] in list_jawaban:
                respon_pola[i] = '?'
                list_jawaban.remove(tebakan[i])
        return ''.join(respon_pola)

        

In [24]:
katla = Katla(daftar_kata)

katla.tebakan_selanjutnya("semua", "***??")
katla.tebakan_selanjutnya("uktab", "?**??")
katla.tebakan_selanjutnya("banyu", "!!!*!")

'bandu'